In [46]:
import pandas as pd
from geopy.distance import lonlat, distance
import numpy as np
from pandas.tseries.offsets import DateOffset

In [66]:
git_hub = "C:\\Users\\Young Jin Park\\Documents\\GitHub\\RGTM\\"
#apt_geocode = pd.read_csv("C:\\Users\\Young Jin Park\\Documents\\GitHub\\RGTM\\Cleaned//merged_manhattan_geocode.csv")

In [61]:
def calculate_x_mile_vals(df, thr, value_col, fn, dist_df=None, merge_df=None):
    values=[]
    
    for i in range(len(df)):
        center = df.loc[i,"GeoCode"]
        sales_date = df.loc[i, "SaleDate"]
        if dist_df is None:
            temp_df = df.copy(deep=True)
        else:
            temp_df = dist_df.copy(deep=True)
        print(i)
        temp_df = temp_df[temp_df["GeoCode"]!=center].copy(deep=True)
        temp_df["Distance"] = temp_df["GeoCode"].apply(lambda x : distance(lonlat(*center), lonlat(*x)).miles)
        print("Calculated Distance")
        temp_df = merge_df.merge(temp_df, how="inner", on ="GeoCode")
        print("Merged Df")
        temp_df[value_col] = temp_df[value_col].apply(lambda x : float(x.replace(",","")))
        print("changed value col type")
        
        if fn == "avg":
            print("getting_into_calc")
            print(len(temp_df[(temp_df["Distance"]<thr)&
                                  (temp_df["SaleDate"]>(sales_date-DateOffset(month=6)))&
                                  (temp_df["SaleDate"]<=sales_date)]))
            values.append(temp_df[(temp_df["Distance"]<thr)&
                                  (temp_df["SaleDate"]>(sales_date-DateOffset(month=6)))&
                                  (temp_df["SaleDate"]<=sales_date)][value_col].mean())
        elif fn == "count":
            values.append(temp_df[temp_df["Distance"]<thr][value_col].count())
        elif fn == "min_dist":
            values.append(temp_df["Distance"].min())
        else:
            print("fn_not defined. Please pass in avg,count or min_dist")
    
    df["Values"] = values
    
    return df
        
        
    
    

In [69]:
apt_df = pd.read_csv(git_hub + "Cleaned\\merged_with_geocode.csv")
apt_df["GeoCode"] = apt_df["GeoCode"].apply(lambda x: eval(x))
apt_df["SaleDate"] = pd.to_datetime(apt_df["SaleDate"])
distinct_apt = pd.read_csv(git_hub + "Cleaned\\merged_manhattan_geocodes.csv")
distinct_apt["GeoCode"] = distinct_apt["GeoCode"].apply(lambda x : eval(x))
merged = pd.read_csv(git_hub + "Cleaned\\FINAL_MERGED_MANHATTAN.csv")
merged["GeoCode"] = merged["GeoCode"].apply(lambda x : eval(x))
merged["SaleDate"] = pd.to_datetime(merged["SaleDate"])

#feature_df = pd.read_csv(git_hub + "Cleaned\\nycopen_zillow_matched_data.csv") 

#distinct_apt["StreetAddress"] = distinct_apt["Address"].apply(lambda x : x.split(",")[0])
#distinct_apt["GeoCode"] = distinct_apt["GeoCode"].apply(lambda x : eval(x))
#apt_df["GeoCode"] = apt_df["GeoCode"].apply(lambda x : eval(x))

#feature_df["StreetAddress"] = feature_df.StreetNum + " " + feature_df.StreetName

#merged = feature_df.merge(distinct_apt[["StreetAddress", "GeoCode"]], how="left", on="StreetAddress")

In [71]:
merged1 = calculate_x_mile_vals(merged, thr=0.5, value_col="SalePrice", fn="avg", dist_df=distinct_apt, merge_df=apt_df)

0
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
1
Calculated Distance
Merged Df
changed value col type
getting_into_calc
799
2
Calculated Distance
Merged Df
changed value col type
getting_into_calc
799
3
Calculated Distance
Merged Df
changed value col type
getting_into_calc
229
4
Calculated Distance
Merged Df
changed value col type
getting_into_calc
271
5
Calculated Distance
Merged Df
changed value col type
getting_into_calc
129
6
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
7
Calculated Distance
Merged Df
changed value col type
getting_into_calc
275
8
Calculated Distance
Merged Df
changed value col type
getting_into_calc
275
9
Calculated Distance
Merged Df
changed value col type
getting_into_calc
316
10
Calculated Distance
Merged Df
changed value col type
getting_into_calc
489
11
Calculated Distance
Merged Df
changed value col type
getting_into_calc
0
12
Calculated Distance
Merged Df
changed value col type
getting_into_calc
281
13

KeyboardInterrupt: 

In [68]:
merged.dropna().to_csv(git_hub+"Cleaned\\FINAL_MERGED_MANHATTAN.csv")